In [2]:
!curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -o input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  1264k      0 --:--:-- --:--:-- --:--:-- 1269k 0  0:00:02 --:--:--  0:00:02  441k


In [3]:
with open('input.txt','r',encoding='utf-8') as f:
    text = f.read()

In [4]:
print('len of dataset in chars:', len(text)) # 1 million chars

len of dataset in chars: 1115394


In [5]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars), vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 65


In [7]:
# tokenize

stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s :  [stoi[c] for c in s] # encoder : take a string, return list on ints
decode = lambda l : ''.join([itos[i] for i in l]) # decode : take a list of ints, output string

print(encode('hi'),decode(encode('hi')))

[46, 47] hi


In [10]:
import torch
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [11]:
n = int(0.9*len(data)) # 90% train , 10% val
train_data = data[:n]
val_data = data[n:]


In [14]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [15]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(context,'-->',target)

tensor([18]) --> tensor(47)
tensor([18, 47]) --> tensor(56)
tensor([18, 47, 56]) --> tensor(57)
tensor([18, 47, 56, 57]) --> tensor(58)
tensor([18, 47, 56, 57, 58]) --> tensor(1)
tensor([18, 47, 56, 57, 58,  1]) --> tensor(15)
tensor([18, 47, 56, 57, 58,  1, 15]) --> tensor(47)
tensor([18, 47, 56, 57, 58,  1, 15, 47]) --> tensor(58)


In [20]:
torch.manual_seed(1337)
batch_size = 4 # how many independent seq will we process in parallel
block_size = 8 # max context len for predictions

def get_batch(split):
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-----')
for b in range(batch_size): # batch dim
    for t in range(block_size): # time dim
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f'when: {context.tolist()} --> {target}')

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----
when: [24] --> 43
when: [24, 43] --> 58
when: [24, 43, 58] --> 5
when: [24, 43, 58, 5] --> 57
when: [24, 43, 58, 5, 57] --> 1
when: [24, 43, 58, 5, 57, 1] --> 46
when: [24, 43, 58, 5, 57, 1, 46] --> 43
when: [24, 43, 58, 5, 57, 1, 46, 43] --> 39
when: [44] --> 53
when: [44, 53] --> 56
when: [44, 53, 56] --> 1
when: [44, 53, 56, 1] --> 58
when: [44, 53, 56, 1, 58] --> 46
when: [44, 53, 56, 1, 58, 46] --> 39
when: [44, 53, 56, 1, 58, 46, 39] --> 58
when: [44, 53, 56, 1, 58, 46, 39, 58] --> 1
when: [52] --> 58
when: [52, 58] --> 1
when: [52, 58, 1] --> 58
when: [52, 58, 1, 58] --> 46
when: [52, 58

In [22]:
print(xb) # input to transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [31]:
# start with bigram model

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        
        # idx and target are both (B,T) tensor of ints
        logits = self.token_embedding_table(idx) # (B,T,C)
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get preds
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:,-1,:] # becomes (B,C)
            # apply softmax to get prob
            probs = F.softmax(logits, dim=-1) # B,C
            # sample
            idx_next = torch.multinomial(probs,num_samples=1) # B,1
            # append sampled idx to running seq
            idx = torch.cat((idx,idx_next), dim=1) # (B,T+1)
        return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m.forward(xb,yb)
print(logits.shape,loss)
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65]) tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [32]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [41]:
batch_size = 32
for steps in range(10000):
    
    # sample a batch of data
    xb, yb = get_batch('train')
    
    # eval the loss
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.576992988586426


In [47]:
print(decode(m.generate(torch.zeros((1,1),dtype=torch.long), max_new_tokens=100)[0].tolist()))


A:
Anck theweag t io, d at.
KE: co ctisefang he t veswerde, t thises;
Bund wiemetiarele hen
le, be a


In [48]:
# self attention

In [55]:
# consider the toy example

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.rand(B,T,C)
x.shape


torch.Size([4, 8, 2])

In [58]:
# want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros(B,T,C)
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev,0)


In [64]:
wei = torch.tril(torch.ones(T,T))
wei /= wei.sum(1,keepdim=True)
xbow2 = wei@x
torch.allclose(xbow,xbow2)

True

In [67]:
# ver 3: use softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros(T,T)
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei@x
torch.allclose(xbow,xbow3)

True

In [54]:
torch.manual_seed(42)
a = torch.tril(torch.ones((3,3))) # lower triangular
a /= a.sum(1,keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c= a @ b
print(a)
print(b)
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [101]:
# Finally soft attention
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.rand(B,T,C)

# single head performs self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B,T,16)
q = query(x) # (B,T,16)
wei = q @ k.transpose(-2,-1) * head_size**-0.5# (B,T,16) @ (B,16,T) -> (B,T,T)

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros(T,T)
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
v = value(x)
out = wei@v

out.shape


torch.Size([4, 8, 16])

In [103]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4852, 0.5148, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3242, 0.3345, 0.3413, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2428, 0.2597, 0.2434, 0.2541, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1960, 0.2043, 0.1992, 0.2105, 0.1899, 0.0000, 0.0000, 0.0000],
        [0.1586, 0.1804, 0.1706, 0.1790, 0.1493, 0.1622, 0.0000, 0.0000],
        [0.1395, 0.1417, 0.1422, 0.1456, 0.1337, 0.1410, 0.1563, 0.0000],
        [0.1203, 0.1272, 0.1256, 0.1299, 0.1166, 0.1240, 0.1248, 0.1316]],
       grad_fn=<SelectBackward0>)

In [105]:
class LayerNorm:
    
    def __init__(self, dim, eps=1e-5):
        self.eps = eps
        # params (trained with backprop)
        self.gamma = torch.ones(dim) # y = BN(gamma) + beta
        self.beta = torch.zeros(dim)
        
    def __call__(self, x):
        # calc the forward pass
        xmean = x.mean(1,keepdim=True) # batch mean
        xvar = x.var(1,keepdim=True) # batch variance
        xhat = (x-xmean)/torch.sqrt(xvar-self.eps) # normalize to unit variance
        self.out = self.gamma * xhat + self.beta # y = BN*gamma + beta
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]
    
